Optuna

In [ ]:
! pip install optuna

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
def create_model(trial: Trial):
    # Hyperparameters to be optimized
    n_layers = trial.suggest_categorical("n_layers", [1, 2, 3])
    n_units = trial.suggest_categorical("n_units", [8, 16, 32, 64, 128])
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_categorical("learning_rate", [0.0001, 0.01, 0.01])
    l1_reg = trial.suggest_categorical("l1_reg", [0.01, 0.05, 0.1])
    l2_reg = trial.suggest_categorical("l2_reg", [0.01, 0.05, 0.1])

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten(), input_shape=(25, 7, 7, 2048)))
    for i in range(n_layers):
        return_sequences = True if i < n_layers - 1 else False
        model.add(LSTM(n_units, return_sequences=return_sequences, kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=BinaryCrossentropy(),
                  metrics=[
                      'accuracy',
                      Precision(name='precision'),
                      Recall(name='recall'),
                      AUC(name='auc'),
                      F1Score()
                  ])

    return model

def objective(trial: Trial):
    model = create_model(trial)
    history = model.fit(
        np.stack(train_features_resnet), np.array(train_labels_resnet),
        validation_data=(np.stack(val_features_resnet), np.array(val_labels_resnet)),
        epochs=50, batch_size=32,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
    )
    # Optimize for validation AUC
    return history.history["val_auc"][-1]

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Results
best_trial = study.best_trial
print(f"Best trial AUC: {best_trial.value}")
print("Best trial params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

### Plotting the Results

In [ ]:
# Predictions on the test data
y_pred = model.predict(np.stack(test_features_resnet))
y_pred_bin = np.where(y_pred > 0.5, 1, 0)

# Confusion matrix
cm = confusion_matrix(np.array(test_labels_resnet), y_pred_bin)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Plotting training & validation loss values per epoch

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot F1 Score
plt.subplot(1, 2, 2)
plt.plot(history.history['f1_score'])
plt.plot(history.history['val_f1_score'])
plt.title('Model F1 Score')
plt.ylabel('F1 Score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Precision and recall Curve
precision, recall, _ = precision_recall_curve(test_labels_resnet, y_pred)
prc_auc = auc(recall, precision)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('test')
model_name = "test"
legend_text = f'{model_name} (AUC = {prc_auc:.2f})'
plt.legend([legend_text])

plt.grid(False)
plt.show()

In [ ]:
#ROC
y_true = test_labels_resnet
y_scores = y_pred

fpr, tpr, thresholds = roc_curve(y_true, y_scores)

roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()